# Exercise

In [11]:
import pandas as pd

In [12]:
housing = pd.read_csv("../../DeepLearning/Tensorflow-Bootcamp-master/02-TensorFlow-Basics/cal_housing_clean.csv")

In [13]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


Separação dos dados em características e respostas

In [19]:
y_val = housing["medianHouseValue"]

In [20]:
x_data = housing.drop("medianHouseValue", axis=1)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_val, test_size=0.3, random_state=101)

Escalonando os valores das features

In [23]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
scaler = MinMaxScaler()

In [25]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

Os valores escalonados precisam ser retraduzidos para pandas dataframe

In [28]:
X_train = pd.DataFrame(data=scaler.transform(X_train), columns=X_train.columns, index=X_train.index)

In [29]:
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

## Criar feature columns do Tensorflow

In [30]:
import tensorflow as tf

In [31]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [32]:
age = tf.feature_column.numeric_column("housingMedianAge")

In [34]:
rooms = tf.feature_column.numeric_column("totalRooms")

In [35]:
bedrooms = tf.feature_column.numeric_column("totalBedrooms")

In [36]:
pop = tf.feature_column.numeric_column("population")

In [38]:
households = tf.feature_column.numeric_column("households")

In [39]:
income = tf.feature_column.numeric_column("medianIncome")

In [40]:
# Agregando todas as features em uma variável
feat_cols = [age, rooms, bedrooms, pop, households, income]

Criar a estimator function para o estimator object

In [79]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,
                                                 y=y_train,
                                                 batch_size=10,
                                                 num_epochs=1000,
                                                 num_threads=4,
                                                 shuffle=True)

Criar o estimator model. DNNRehressor

In [69]:
model = tf.estimator.DNNRegressor(hidden_units=[4, 6, 8, 8, 6, 4], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpmvrp8ya5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3e47c1b160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Treinar o modelo para 1000 passos

In [70]:
model.train(input_fn=input_func, steps=200000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpmvrp8ya5/model.ckpt.
INFO:tensorflow:loss = 293128470000.0, step = 1
INFO:tensorflow:global_step/sec: 543.866
INFO:tensorflow:loss = 149730400000.0, step = 101 (0.186 sec)
INFO:tensorflow:global_step/sec: 706.979
INFO:tensorflow:loss = 55763590000.0, step = 201 (0.140 sec)
INFO:tensorflow:global_step/sec: 696.704
INFO:tensorflow:loss = 58027720000.0, step = 301 (0.144 sec)
INFO:tensorflow:global_step/sec: 707.328
INFO:tensorflow:loss = 56663230000.0, step = 401 (0.141 sec)
INFO:tensorflow:global_step/sec: 697.074
INFO:tensorflow:loss = 154348570000.0, step = 501 (0.144 sec)
INFO:tensorflow:global_step/sec: 658.585
INFO:tensorflow:loss = 50980446000.0, step = 601 (0.152 sec)
INFO:tensorflow:global_ste

Criar uma prediction input function e use o método .predict do seu estimator model para criar uma lista de predições no seu test data

In [71]:
# Não uso um valor de y porque estou agora prevendo os resultados
# Também preciso apenas de 1 epoch
predic_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,
                                                       batch_size=10,
                                                       num_epochs=1,
                                                       shuffle=False)

In [72]:
pred_gen = model.predict(predic_input_func)

In [73]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpmvrp8ya5/model.ckpt-200000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [74]:
predictions

[{'predictions': array([296504.56], dtype=float32)},
 {'predictions': array([577967.06], dtype=float32)},
 {'predictions': array([157969.75], dtype=float32)},
 {'predictions': array([188527.98], dtype=float32)},
 {'predictions': array([471344.88], dtype=float32)},
 {'predictions': array([173907.66], dtype=float32)},
 {'predictions': array([207663.6], dtype=float32)},
 {'predictions': array([183829.5], dtype=float32)},
 {'predictions': array([263130.88], dtype=float32)},
 {'predictions': array([315706.], dtype=float32)},
 {'predictions': array([199270.4], dtype=float32)},
 {'predictions': array([200544.69], dtype=float32)},
 {'predictions': array([165061.53], dtype=float32)},
 {'predictions': array([156257.33], dtype=float32)},
 {'predictions': array([312620.12], dtype=float32)},
 {'predictions': array([188357.67], dtype=float32)},
 {'predictions': array([158244.17], dtype=float32)},
 {'predictions': array([199685.66], dtype=float32)},
 {'predictions': array([181585.34], dtype=float32)}

Calcular o RMSE

In [75]:
final_preds = []

for pred in predictions:
    final_preds.append(pred["predictions"])

In [76]:
from sklearn.metrics import mean_squared_error

In [77]:
mean_squared_error(y_test, final_preds)**0.5

78388.20235233082

In [78]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000
